Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [17]:
from itertools import product
from random import random, randint, choice, seed, choices
import numpy as np
from scipy import sparse
from copy import  copy
from functools import reduce

In [18]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets

# Halloween Challenge

Find the best solution with the fewest calls to the fitness functions for:

* `num_points = [100, 1_000, 5_000]`
* `num_sets = num_points`
* `density = [.3, .7]`

In [19]:
def fitness1(sets, state):
    cost = sum(state)
    valid = np.all(
        reduce(
            np.logical_or,
            [sets[i, :] for i, t in enumerate(state) if t],
            np.array([False for _ in range(sets.shape[1])]),
        )
    )
    return valid, -cost

def fitness2(sets, state):
    cost = sum(state)
    if np.array(state).any():
        valid = sets[np.array(state), :].max(axis=0).sum()
    else:
        valid = 0
    return valid, -cost

fitness = fitness2

In [20]:
def tweak(state, size):
    new_state = copy(state)
    index = randint(0, size - 1)
    new_state[index] = not new_state[index]
    return new_state

In [25]:
NUM_SETS = 1_000
NUM_NODES = 1_000
sets = make_set_covering_problem(NUM_SETS, NUM_NODES, .3).toarray()

<h1 align="center">Standard Hill Climbing<h1>

In [ ]:
current_state = [choice([False, False, False, False, False, False]) for _ in range(NUM_SETS)]
print(fitness(sets, current_state))
fitness_calls = 0
# visited map
visited = dict()

for step in range(10_000):
    new_state = tweak(current_state, NUM_SETS)
    curr_tuple = tuple(current_state)
    new_tuple = tuple(new_state)
    if curr_tuple in visited:
        s = visited[curr_tuple]
    else:
        s = fitness(sets, current_state)
        fitness_calls += 1
        visited[curr_tuple] = s
    if new_tuple in visited:
        s_new = visited[new_tuple]
    else:
        s_new = fitness(sets, new_state)
        fitness_calls += 1
        visited[new_tuple] = s_new
    if s_new >= s:
        current_state = new_state
        print(fitness(sets, current_state))
print(fitness_calls)

<h1 align="center">Simulated Annealing Hill Climbing<h1>

In [ ]:
# parameters
T = 1
Tmin = 0.0001
alpha = 0.95
numIterations = 220
# visited map
visited = dict()

global_min = ()
s_min = (0, 0)
fitness_calls = 0
while T >= Tmin:
    for step in range(numIterations):
        new_state = tweak(current_state, NUM_SETS)
        curr_tuple = tuple(current_state)
        new_tuple = tuple(new_state)
        if curr_tuple in visited:
            s = visited[curr_tuple]
        else:
            s = fitness(sets, current_state)
            fitness_calls += 1
            visited[curr_tuple] = s
        if new_tuple in visited:
            s_new = visited[new_tuple]
        else:
            s_new = fitness(sets, new_state)
            fitness_calls += 1
            visited[new_tuple] = s_new
        if s_new >= s:
            current_state = new_state
            if s_new > s_min:
                global_min = new_state
                s_min = s_new
        else:
             p = np.exp(-(sum(s) - sum(s_new)) / T)
             current_state = choices([current_state, new_state], weights=(1 - p, p), k=1)[0]
    T *= alpha  # Decreases T, cooling phase

print(s_min)
print(fitness_calls)